<a href="https://colab.research.google.com/github/linah-kg/Deep-Learning-Based-Shoplifting-Detection/blob/main/Model_Training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import cv2
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import seaborn as sns
from tensorflow.keras.applications import MobileNetV3Small
from tensorflow.keras.layers import GRU, Dense, BatchNormalization, Flatten, TimeDistributed, GlobalAveragePooling2D
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, confusion_matrix, roc_curve, auc
import time
import pickle
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, LearningRateScheduler, ReduceLROnPlateau


In [ ]:
SEQUENCE_LENGTH = 30
import pickle
from google.colab import drive
import os

drive.mount('/content/drive')

data_path = '/content/drive/My Drive/Shoplifting-Datasets/processed_data_no_augmentation_splited801010'

def load_data(file_name):
    with open(file_name, 'rb') as file:
        return pickle.load(file)

# Load the data
train_data = load_data(os.path.join(data_path, f'train_data_{SEQUENCE_LENGTH}.pkl'))
val_data = load_data(os.path.join(data_path, f'val_data_{SEQUENCE_LENGTH}.pkl'))
test_data = load_data(os.path.join(data_path, f'test_data_{SEQUENCE_LENGTH}.pkl'))

# Unpack the datasets
X_train, y_train = train_data
X_val, y_val = val_data
X_test, y_test = test_data

print("Data loaded successfully!")

In [ ]:
NUM_CLASSES = 2
# Ensure labels are in one-hot encoded format
y_train = to_categorical(y_train, NUM_CLASSES)
y_val = to_categorical(y_val, NUM_CLASSES)
y_test = to_categorical(y_test, NUM_CLASSES)


print(f"y_train shape: {y_train.shape}")
print(f"y_val shape: {y_val.shape}")
print(f"y_test shape: {y_test.shape}")

In [ ]:
import tensorflow as tf
from tensorflow.keras import mixed_precision
from tensorflow.keras.layers import (Input, GRU, Bidirectional, Dense, Concatenate,
                                     TimeDistributed, Dropout, BatchNormalization,
                                     GlobalAveragePooling2D, Reshape, Multiply, Softmax)
from tensorflow.keras.applications import MobileNetV2, EfficientNetB0
from tensorflow.keras.models import Model
from keras.saving import register_keras_serializable
mixed_precision.set_global_policy('mixed_float16')


frame_shape = (224, 224, 3)
num_frames = 30
input_shape = (num_frames, 224, 224, 3)


def FeatureAttention(x):
    squeeze = TimeDistributed(GlobalAveragePooling2D())(x)
    excitation = Dense(units=squeeze.shape[-1] // 16, activation='relu')(squeeze)
    excitation = Dense(units=squeeze.shape[-1], activation='sigmoid')(excitation)
    excitation = TimeDistributed(Reshape((1, 1, -1)))(excitation)
    return Multiply()([x, excitation])

@register_keras_serializable()
class TemporalAttention(tf.keras.layers.Layer):
    def __init__(self, **kwargs):
        super(TemporalAttention, self).__init__(**kwargs)
        self.dense = Dense(1)

    def call(self, inputs):
        score = tf.nn.tanh(self.dense(inputs))
        attention_weights = Softmax(axis=1)(score)
        context_vector = attention_weights * inputs
        return tf.reduce_sum(context_vector, axis=1)

    def get_config(self):
        config = super(TemporalAttention, self).get_config()
        return config

# Input Layer
input_layer = Input(shape=input_shape)

# Feature Extractors: Two parallel light CNNs
mobilenet = MobileNetV2(weights='imagenet', include_top=False, input_shape=frame_shape)
efficientnet = EfficientNetB0(weights='imagenet', include_top=False, input_shape=frame_shape)

# Freeze all layers and unfreeze the last 40 layers for fine-tuning
mobilenet.trainable = False
efficientnet.trainable = False
for layer in mobilenet.layers[-40:]:
    layer.trainable = True
for layer in efficientnet.layers[-40:]:
    layer.trainable = True

# Apply CNNs to each frame using TimeDistributed
mobilenet_out = TimeDistributed(mobilenet)(input_layer)
efficientnet_out = TimeDistributed(efficientnet)(input_layer)

# Apply Feature Attention
mobilenet_out = FeatureAttention(mobilenet_out)
efficientnet_out = FeatureAttention(efficientnet_out)

# Global Average Pooling and feature concatenation
mobilenet_out = TimeDistributed(GlobalAveragePooling2D())(mobilenet_out)
efficientnet_out = TimeDistributed(GlobalAveragePooling2D())(efficientnet_out)
merged_features = Concatenate()([mobilenet_out, efficientnet_out])

# Temporal modeling with Two stacked Bidirectional GRU layers
gru_out1 = Bidirectional(GRU(256, return_sequences=True))(merged_features)
gru_out1 = Dropout(0.3)(gru_out1)
gru_out2 = Bidirectional(GRU(256, return_sequences=True))(gru_out1)
gru_out2 = TemporalAttention()(gru_out2)

# Fully Connected Layers with BatchNorm & Dropout
dense1 = Dense(512, activation='relu')(gru_out2)
dense1 = BatchNormalization()(dense1)
dense1 = Dropout(0.3)(dense1)
dense2 = Dense(256, activation='relu')(dense1)
dense2 = BatchNormalization()(dense2)
dense2 = Dropout(0.3)(dense2)
dense3 = Dense(128, activation='relu')(dense2)
dense3 = BatchNormalization()(dense3)
dense3 = Dropout(0.3)(dense3)

# Final output layer
output_layer = Dense(2, activation='softmax', dtype='float32')(dense3)


model = Model(inputs=input_layer, outputs=output_layer)
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

model.summary()


In [ ]:


filepath = '/content/drive/My Drive/Shoplifting-Datasets/Best-Models/PrallelCNN-MobileNetV2-EfficientNetB0-2BiGRU_model_epoch_{epoch:02d}_val_acc_{val_accuracy:.2f}.keras'

checkpoint_callback = ModelCheckpoint(
    filepath=filepath,
    monitor='val_accuracy',
    save_best_only=True,
    mode='max',
    verbose=1
)


early_stopping = EarlyStopping(
    monitor='val_accuracy',
    patience=20,
    restore_best_weights=True
)

history = model.fit(X_train, y_train, epochs=50, batch_size=8, validation_data=(X_val, y_val), callbacks=[checkpoint_callback, early_stopping])

# Evaluate model
y_pred = model.predict(X_test)
y_pred_classes = np.argmax(y_pred, axis=1)
y_true = np.argmax(y_test, axis=1)

accuracy = accuracy_score(y_true, y_pred_classes)
precision = precision_score(y_true, y_pred_classes)
recall = recall_score(y_true, y_pred_classes)
f1 = f1_score(y_true, y_pred_classes)

print(f"Test Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1-Score: {f1:.4f}")

# Confusion Matrix function
def plot_confusion_matrix(y_true, y_pred_classes):
    cm = confusion_matrix(y_true, y_pred_classes)
    plt.figure(figsize=(6, 5))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=['Normal', 'Shoplifting'], yticklabels=['Normal', 'Shoplifting'])
    plt.xlabel('Predicted Label')
    plt.ylabel('True Label')
    plt.title('Confusion Matrix')
    plt.show()

# Plot Confusion Matrix
plot_confusion_matrix(y_true, y_pred_classes)

